# DAILY SQL PROGRESS LOG 

To be used with notion. Contains daily sql questions / repetitions.


## January 15, 2026

In [ ]:
-- Histogram of Tweets - Easy 
WITH TWEETER AS(
SELECT 
  count(tweet_id) as tweet_bucket,  
  user_id
FROM tweets
WHERE EXTRACT (YEAR FROM tweet_date) = 2022
GROUP BY user_id
)
SELECT 
  tweet_bucket, 
  COUNT(user_id) AS users_num
FROM TWEETER
GROUP BY tweet_bucket

In [ ]:
-- Fill missing client data
with cte as(
SELECT *, 
  Count(category) OVER (order by product_id) as category_number
  
FROM products)

SELECT product_id, 
  COALESCE(category, MAX(category) OVER (PARTITION BY category_number
                                          ORDER BY product_id)), 
  name
  
FROM cte;

# 19th January, 2026 

In [ ]:
WITH CTE AS (
SELECT
  user_id, 
  row_number()over(partition by user_id order by transaction_date) AS transact,
  transaction_date,
  spend 
FROM transactions
)
SELECT 
  user_id, 
  spend, 
  transaction_date
FROM CTE 
WHERE transact=3

# 20th January. 2026

In [ ]:
SELECT 
  EXTRACT (MONTH FROM event_date) as mth, 
  COUNT( DISTINCT user_id) as monthly_active_users 
FROM user_actions 
WHERE user_id IN (
                  SELECT DISTINCT user_id 
                  FROM user_actions 
                  WHERE EXTRACT(MONTH FROM event_date)=6
                  AND event_type IN ('sign-in','comment','like')
                  )
AND EXTRACT(MONTH FROM event_date) = 7
AND event_type IN ('sign-in','comment','like')
GROUP BY mth

In [ ]:
WITH YEAR_DATA AS(
SELECT 
  EXTRACT (YEAR FROM transaction_date) as yr, 
  product_id, 
  SUM(spend) OVER (
                    PARTITION BY product_id, transaction_date
                  ) AS curr_year_spend, 
  LAG(SUM(spend))OVER (
                        PARTITION BY product_id
                      ) AS prev_year_spend
FROM user_transactions 
GROUP BY  transaction_date, product_id, spend
)
SELECT 
  yr, 
  product_id, 
  curr_year_spend,
  prev_year_spend,
  ROUND(((curr_year_spend - prev_year_spend)::DECIMAL/prev_year_spend)*100,2) AS yoy_rate
  FROM YEAR_DATA